# Local ColabNAS - Neural Architecture Search for TinyML

This notebook demonstrates how to use ColabNAS locally on your machine for finding optimal CNN architectures for TinyML devices.

## Key Features:
- Runs locally without Google Colab dependencies
- Optimized for resource-constrained devices
- Automatic quantization to uint8
- Memory usage estimation
- Early stopping for faster convergence

## 1. Setup and Imports

In [1]:
import tensorflow as tf
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from local_colabnas import LocalColabNAS, test_tflite_model

# Check TensorFlow version and GPU availability
print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Set memory growth for GPU (if available)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

TensorFlow version: 2.20.0
GPU available: []


## 2. Download and Prepare Dataset

We'll use the flower dataset as an example. You can replace this with your own dataset.

In [2]:
# Download flower dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = Path(data_dir).with_suffix('')

print(f"Dataset downloaded to: {data_dir}")
print(f"Classes found: {[d.name for d in data_dir.iterdir() if d.is_dir()]}")

Dataset downloaded to: C:\Users\Chakkaphan\.keras\datasets\flower_photos_extracted
Classes found: ['flower_photos']


## 3. Configure Target Device Constraints

Define the hardware constraints for your target TinyML device.

In [3]:
# Input image configuration
input_shape = (64, 64, 3)  # Slightly larger for better accuracy

# Target device: STM32L412KBU3 (example)
# 273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40 * 1024      # 40 KB
Flash_upper_bound = 128 * 1024        # 128 KB  
MACC_upper_bound = 273 * 10000        # CoreMark * 1e4

# Training configuration
val_split = 0.2
cache = False  # Set to True if you have enough RAM
save_path = Path('./nas_results')
save_path.mkdir(exist_ok=True)

print(f"Target constraints:")
print(f"  RAM: {peak_RAM_upper_bound/1024:.1f} KB")
print(f"  Flash: {Flash_upper_bound/1024:.1f} KB")
print(f"  MACC: {MACC_upper_bound:,}")
print(f"  Input shape: {input_shape}")

Target constraints:
  RAM: 40.0 KB
  Flash: 128.0 KB
  MACC: 2,730,000
  Input shape: (64, 64, 3)


## 4. Initialize LocalColabNAS

In [4]:
# Initialize the NAS system
nas = LocalColabNAS(
    max_RAM=peak_RAM_upper_bound,
    max_Flash=Flash_upper_bound, 
    max_MACC=MACC_upper_bound,
    path_to_training_set=str(data_dir),
    val_split=val_split,
    cache=cache,
    input_shape=input_shape,
    save_path=str(save_path)
)

print("LocalColabNAS initialized successfully!")

Initializing LocalColabNAS with 1 classes
Found 3670 files belonging to 1 classes.
Using 2936 files for training.
Found 3670 files belonging to 1 classes.
Using 734 files for validation.
LocalColabNAS initialized successfully!


## 5. Run Neural Architecture Search

This will automatically search for the best architecture within your constraints.

In [ ]:
# Start the search process
print("Starting Neural Architecture Search...")
print("This may take several minutes to hours depending on your hardware.")

best_model_path = nas.search()

if best_model_path:
    print(f"\n✅ Search completed successfully!")
    print(f"Best model saved at: {best_model_path}")
else:
    print("\n❌ No feasible architecture found within the given constraints.")
    print("Consider relaxing the constraints or using a smaller input size.")

Starting Neural Architecture Search...
This may take several minutes to hours depending on your hardware.
Starting NAS search at 2026-01-05 16:36:24.386121

k_4_c_0



c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\keras\src\ops\nn.py:944: UserWarning: You are using a softmax over axis -1 of a tensor of shape (None, 1). This axis has size 1. The softmax operation will always return the value 1, which is likely not what you intended. Did you mean to use a sigmoid instead?
  warnings.warn(
c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\keras\src\losses\losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


92/92 ━━━━━━━━━━━━━━━━━━━━ 6s 49ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00



RAM: 49844, Flash: 57792, MACC: 442385




{'k': 4, 'c': 0, 'RAM': 'Outside the upper bound', 'Flash': 57792, 'MACC': 442385, 'max_val_acc': -3}




k_8_c_0

92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00



RAM: 50644, Flash: 62008, MACC: 884801




{'k': 8, 'c': 0, 'RAM': 'Outside the upper bound', 'Flash': 62008, 'MACC': 884801, 'max_val_acc': -3}




k_2_c_0

92/92 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00



RAM: 49492, Flash: 57912, MACC: 221189




{'k': 2, 'c': 0, 'RAM': 'Outside the upper bound', 'Flash': 57912, 'MACC': 221189, 'max_val_acc': -3}




k_1_c_0



## 6. Test the Found Architecture

Let's test the performance of the discovered model.

In [ ]:
if best_model_path and best_model_path.exists():
    # Create a test dataset
    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=str(data_dir),
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb' if input_shape[2] == 3 else 'grayscale',
        batch_size=32,
        image_size=input_shape[0:2],
        shuffle=True,
        seed=42,
        validation_split=0.8,  # Use 20% for testing
        subset='validation'
    )
    
    # Test the model
    print("Testing the discovered model...")
    accuracy = test_tflite_model(best_model_path, test_ds)
    
    # Get model size
    model_size = best_model_path.stat().st_size
    print(f"Model size: {model_size/1024:.2f} KB")
    print(f"Model path: {best_model_path}")
else:
    print("No model to test - search was unsuccessful.")

## 7. Manual Architecture Testing (Optional)

You can also test specific architectures manually.

In [ ]:
# Test a specific architecture manually
def test_manual_architecture(k, c):
    """Test a specific architecture configuration"""
    print(f"Testing architecture: k={k}, c={c}")
    
    model, macc, cells_limited = nas.Model(k, c)
    model.summary()
    
    print(f"MACC: {macc:,}")
    print(f"Cells limited: {cells_limited}")
    print(f"Total parameters: {model.count_params():,}")
    
    return model

# Example: test a small architecture
# test_model = test_manual_architecture(k=4, c=2)

## 8. Deployment Information

Information about deploying your model to actual hardware.

In [ ]:
print("🚀 Deployment Information:")
print("\n1. Your quantized TFLite model is ready for deployment")
print("2. The model uses uint8 quantization for optimal performance")
print("3. For STM32 deployment, you can use STM32Cube.AI")
print("4. For other microcontrollers, use TensorFlow Lite Micro")

if best_model_path and best_model_path.exists():
    print(f"\n📁 Model file: {best_model_path}")
    print(f"📊 Model size: {best_model_path.stat().st_size/1024:.2f} KB")
    
    # Load and inspect the model
    interpreter = tf.lite.Interpreter(str(best_model_path))
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    
    print(f"\n🔧 Model Details:")
    print(f"  Input shape: {input_details['shape']}")
    print(f"  Input type: {input_details['dtype']}")
    print(f"  Output shape: {output_details['shape']}")
    print(f"  Output type: {output_details['dtype']}")
    
    if input_details['dtype'] == tf.uint8:
        scale, zero_point = input_details['quantization']
        print(f"  Input quantization - Scale: {scale:.6f}, Zero point: {zero_point}")

## 9. Visualization (Optional)

Visualize some sample predictions.

In [ ]:
def visualize_predictions(model_path, dataset, num_samples=6):
    """Visualize model predictions on sample images"""
    if not model_path or not model_path.exists():
        print("No model available for visualization")
        return
    
    interpreter = tf.lite.Interpreter(str(model_path))
    interpreter.allocate_tensors()
    
    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    
    # Get class names
    class_names = [d.name for d in data_dir.iterdir() if d.is_dir()]
    
    plt.figure(figsize=(15, 10))
    
    sample_count = 0
    for batch in dataset.take(1):
        images, labels = batch
        
        for i in range(min(num_samples, images.shape[0])):
            image = images[i:i+1]
            true_label = labels[i].numpy().argmax()
            
            # Prepare input
            if input_details['dtype'] == tf.uint8:
                scale, zero_point = input_details['quantization']
                image_input = image / scale + zero_point
                image_input = tf.cast(image_input, tf.uint8)
            else:
                image_input = tf.cast(image, input_details['dtype'])
            
            # Make prediction
            interpreter.set_tensor(input_details['index'], image_input)
            interpreter.invoke()
            prediction = interpreter.get_tensor(output_details['index'])
            predicted_label = prediction.argmax()
            confidence = prediction.max()
            
            # Plot
            plt.subplot(2, 3, i + 1)
            plt.imshow(image[0].numpy().astype(np.uint8))
            plt.title(f'True: {class_names[true_label]}\n'
                     f'Pred: {class_names[predicted_label]}\n'
                     f'Conf: {confidence:.2f}')
            plt.axis('off')
            
            sample_count += 1
            if sample_count >= num_samples:
                break
        
        if sample_count >= num_samples:
            break
    
    plt.tight_layout()
    plt.show()

# Visualize predictions if model exists
if best_model_path and best_model_path.exists():
    visualize_predictions(best_model_path, test_ds)
else:
    print("No model available for visualization")